В данном ноутбуке для сравнения были сделаны несколько тривиальных решений, для сравнения в качестве базового варианта

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
from subprocess import check_output
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [2]:
order_products_train_df = pd.read_csv("../data/input/order_products__train.csv")
order_products_prior_df = pd.read_csv("../data/input/order_products__prior.csv")
orders_df = pd.read_csv("../data/input/orders.csv")
products_df = pd.read_csv("../data/input/products.csv")
aisles_df = pd.read_csv("../data/input/aisles.csv")
departments_df = pd.read_csv("../data/input/departments.csv")

разбиение на train, val.

In [3]:
train_ids, val_ids = train_test_split(orders_df.query('eval_set == "train"')['user_id'].unique(), test_size = 0.3, random_state = 1)
train_ids, val_ids = set(train_ids), set(val_ids)

вспомогательные функции

In [4]:
def f1_score(true_list, pred_list):
    if true_list == [] and pred_list == []:
        raise ValueError((true_list, pred_list)) 
    true = set(true_list)
    pred = set(pred_list)
    cross_size = len(true & pred)
    if cross_size == 0:
        return 0
    p = 1. * cross_size / len(pred)
    r = 1. * cross_size / len(true)
    return 2 * p * r / (p + r)

def get_basket_dict(df):
    d = defaultdict(list)
    for product, order_id in zip(df['product_id'], df['user_id']):
        d[order_id].append(product)
    return d

def in_val(x):
    if x in train_ids:
        return 0
    elif x in val_ids:
        return 1
    else:
        raise ValueError('wrong id')

def score_baskets(pred_baskets, true_baskets):
    scores = [f1_score(products, pred_baskets[order]) for order, products in true_baskets.items()]
    return np.mean(scores)

создаем словари вида user_id: (list of product_id) для того, чтобы можно было скоринг сделать

In [5]:
temp = order_products_train_df.merge(orders_df[['order_id', 'user_id']], how = 'inner', on = ['order_id'])
temp['is_val'] = temp['user_id'].apply(in_val)

train_baskets = get_basket_dict(temp.query('is_val == 0'))
val_baskets = get_basket_dict(temp.query('is_val == 1'))

### Базовое решение 1.
Покупатели повторяют все, что раньше заказывали.

In [6]:
history = order_products_prior_df.merge(orders_df.query('eval_set == "prior"')[['order_id', 'user_id']], 
                                        how = 'inner', on = ['order_id'])

baseline_1_prediction = get_basket_dict(history)

print('Train: ', score_baskets(baseline_1_prediction, train_baskets))
print('Val: ', score_baskets(baseline_1_prediction, val_baskets))

Train:  0.19757637893447272
Val:  0.19728520196853316


### Базовое решение 2.
Покупатели повторяют все, что перезаказываемое из истории заказов.

In [7]:
baseline_2_prediction = get_basket_dict(history.query('reordered == 1'))

print('Train: ', score_baskets(baseline_2_prediction, train_baskets))
print('Val: ', score_baskets(baseline_2_prediction, val_baskets))

Train:  0.2512355052520194
Val:  0.2499764993808088


### Базовое решение 3.
Покупатели повторяют прошлую покупку.

In [8]:
prev_order = orders_df.sort_values(['user_id', 'order_number'])\
                      .query('eval_set == "prior"')\
                      .drop_duplicates(subset=['user_id'], keep='last')[['order_id', 'user_id']]

prev_basket = order_products_prior_df.merge(prev_order, how = 'inner', on = ['order_id'])
prev_basket.head(5)

baseline_3_prediction = get_basket_dict(prev_basket)

print('Train: ', score_baskets(baseline_3_prediction, train_baskets))
print('Val: ', score_baskets(baseline_3_prediction, val_baskets))

Train:  0.26150811054948786
Val:  0.2619116555459477


### Базовое решение 4.
Покупатели повторяют прошлую покупку, но при этом оставляют только то, что раньше заказывали.

In [9]:
prev_basket_reordered = order_products_prior_df.merge(prev_order, how = 'inner', on = ['order_id']).query('reordered == 1')
prev_basket_reordered.head(5)

baseline_4_prediction = get_basket_dict(prev_basket_reordered)

print('Train: ', score_baskets(baseline_4_prediction, train_baskets))
print('Val: ', score_baskets(baseline_4_prediction, val_baskets))

Train:  0.24707796891157688
Val:  0.24737140721780046
